# Program Overview



# Headers

In [1]:
import header
paths = header.setup_environment()
for k,v in paths.items():
    print(k, '->', v)
    

Darwin
HOME -> /Users/thompsong
Developer -> /Users/thompsong/Developer
repodir -> /Users/thompsong/Developer/KSCRocketSeismoHydrology/Python/new_workflow
work -> /Users/thompsong/work
local_outdir -> /Users/thompsong/work/PROJECTS/KSC_EROSION
DROPBOX_TOP -> /Users/thompsong/Dropbox
new_data -> /Users/thompsong/data/KSCwell
DROPBOX_DATA_TOP -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA
dropbox_outdir -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/new_workflow
WELLDATA_TOP -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData
TOB3_DIR -> /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/Uploads


In [ ]:
import os
import obspy
import glob
import pandas as pd
import sys
sys.path.append('..')
import libWellData as LLE
print(os.getcwd())
os.chdir('../..')
transducersDF = LLE.get_transducers_dataframe(paths)
os.chdir('/data/KSC/EROSION/obsolete/miniseed/good')
print(transducersDF)

# Functions

In [ ]:
def Stream2csv(st, csvfile):
    stime = min([tr.stats.starttime for tr in st])
    etime = max([tr.stats.endtime for tr in st])
    st.trim(starttime=stime, endtime=etime, pad=True, fill_value=0.0)
    dt = st[0].times('utcdatetime')
    df = pd.DataFrame()
    df['TIMESTAMP'] = dt
    for tr in st:
        serialnum = tr.stats.station + tr.stats.location
        df[serialnum] = tr.data
    print(df.head())

    LLE.uncalibrate_to_raw(transducersDF, df, csvfile)


# Main program

In [ ]:
allfiles = sorted(glob.glob('*.ms'))
lod = []
for msfile in allfiles:
    parts = msfile.split('.')
    network = parts[0]
    station = parts[1] + parts[2]
    channel = parts[3]
    stime = obspy.UTCDateTime.strptime(parts[4], '%Y%m%d_%H%M%S').datetime
    etime = obspy.UTCDateTime.strptime(parts[5], '%Y%m%d_%H%M%S').datetime
    if channel[0]=='H':
        fsamp=100
    elif channel[0]=='B':
        fsamp=20
    elif channel[0]=='L':
        fsamp=1
 
    mydict = {'stime':stime, 'etime':etime, 'well':network, 'serialnum':station,'channel':channel,'msfile':msfile,'fsamp':fsamp}
    lod.append(mydict)
df = pd.DataFrame(lod)
#print(df)
df = df.sort_values(by=['stime'])
#print(df.head())
#dfg=df.groupby(['stime', 'fsamp'], as_index=False).size()
dfg=df.groupby(['stime', 'fsamp'])
for name,group in dfg:
    outfile=f'../Stream_{name[0].strftime("%Y%m%d%H%M%S")}_{name[1]}Hz.mseed'
    stall = obspy.Stream()
    print(name)
    #print(group)
    for i, row in group.iterrows():
        #print(row)
        st = obspy.read(row['msfile'])
        st.merge()
        for tr in st:
            stall.append(tr)
    stall.write(outfile,format='MSEED')
    Stream2csv(stall, outfile.replace('.mseed','.csv'))
